In [1]:
import torch
import mlflow
import json
import datetime
import head_model
from transformers import AutoTokenizer, AutoConfig, AutoModel
from transformers import MLukeTokenizer, LukeConfig, LukeModel

### Training and Tracking

In [ ]:
from mlflow.models.signature import infer_signature
mlflow.set_tracking_uri("http://127.0.0.1:4000/")
mlflow.set_experiment("Ner-Experiment")

In [2]:
with open("config.json") as f:
    config = json.load(f)

In [3]:
tag_values = ['PAD', 'ADDRESS', 'EMAIL','PERSON','PHONENUMBER','MISCELLANEOUS','PERSONTYPE',
              'ORGANIZATION','PRODUCT','IP','LOCATION','O','DATETIME', 'URL']
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
print(device)

cpu


In [4]:
model_names = ['xlmr_linear', 'xlmr_crf', 'mluke_linear', 'mluke_crf']

In [5]:
config['models']['xlmr_linear']['pre_train']

'xlm-roberta-base'

In [6]:
metrics = config['models'][model_names[0]]['metrics']

In [7]:
metrics["f2_span_train"]

97.47

In [8]:
for model_name in model_names:
    if "xlmr" in model_name:
        tokenizer = AutoTokenizer.from_pretrained(config['models'][model_name]['pre_train'], do_lower_case=False,use_fast=False)
        model_config = AutoConfig.from_pretrained(config['models'][model_name]['pre_train'], output_hidden_states=True)
        model_config.hidden_dropout_prob = config['models'][model_name]['params']['bert_hidd_dropout']
        model_config.attention_probs_dropout_prob = config['models'][model_name]['params']['bert_att_dropout']
        base_model = AutoModel.from_pretrained(config['models'][model_name]['pre_train'], config=model_config)
    elif "mluke" in model_name:
        tokenizer = MLukeTokenizer.from_pretrained(config['models'][model_name]['pre_train'], do_lower_case=False,use_fast=False)
        model_config = LukeConfig.from_pretrained(config['models'][model_name]['pre_train'], output_hidden_states=True)
        model_config.hidden_dropout_prob = config['models'][model_name]['params']['bert_hidd_dropout']
        model_config.attention_probs_dropout_prob = config['models'][model_name]['params']['bert_att_dropout']
        base_model = LukeModel.from_pretrained(config['models'][model_name]['pre_train'], config=model_config)

    if "linear" in model_name:
        model = head_model.BaseBertSoftmax(model=base_model, drop_out=config['models'][model_name]['params']['linear_dropout'], num_labels=len(tag_values))
        model.to(device)
    elif "crf" in model_name:
        model = head_model.BaseBertCrf(model=base_model, drop_out=config['models'][model_name]['params']['linear_dropout'], num_labels=len(tag_values))
        model.to(device)
    params = config['models'][model_name]['params']
    tag = {"data": config['data'], "model": model_name}
    runname = model_name + str(datetime.datetime.now()).replace(" ","T")
    with mlflow.start_run(run_name=runname) as run:
        mlflow.set_tags(tag)                                    # Tags to help in tracking

        metrics = config['models'][model_name]['metrics']
        mlflow.log_params(params)                               # Log params/hyperparameters used in experiement
        
        mlflow.log_metric("f2_span_train", metrics["f2_span_train"])
        mlflow.log_metric("f2_span_dev", metrics["f2_span_dev"])
        mlflow.log_metric("f2_span_test", metrics["f2_span_test"])
        
        #signature = infer_signature(X, model.predict(X))
        mlflow.pytorch.log_model(model, artifact_path="models")
        #mlflow.sklearn.log_model(model, artifact_path="models", signature=signature) # Log model created
    mlflow.end_run()

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2022/02/14 11:22:59 WARNING mlflow.utils.requirements_utils: Found torch version (1.10.2+cu113) contains a local version label (+cu113). MLflow logged a pip requirement for this package as 'torch==1.10.2' without the local version label to make it installable from PyPI. To specify pi